In [1]:
using Random, Plots, Distances

In [2]:
Random.seed!(2025);

In [3]:
fixed_colors = [:tomato, :mediumseagreen, :steelblue]

function show_grid(t, opinions, font)
    mapped_values = [op == 1 ? 1 : (op == 2 ? 2 : 3) for op in opinions]

    p = heatmap(mapped_values, color=fixed_colors, xticks=:none, yticks=:none,
                clim=(1,3), legend=false, frame=false, yflip=true, title="t=$t",
                size=(500, 500))

    counter = 1
    for i in 1:size(opinions, 1), j in 1:size(opinions, 2)
        annotate!(j, i, text(string(counter), :black, :center, font))
        counter += 1
    end
    display(p)
end;

In [4]:
function NSL_model(;L=nothing, alpha=nothing, zad=nothing)
    N = L^2
    t = 0

    if (zad == 1)
        opinions = [1 2 1;
                    1 3 3;
                    1 2 3];
        agents_id = [1 2 3;
                     4 5 6;
                     7 8 9];
        agents_properties = permutedims(reshape([(i/10, 1 - i/10)[k] for i in 1:N, k in 1:2], 3, 3, 2), (2, 1, 3))
    elseif (zad == 2)
        opinions = rand(1:3, L, L)
        agents_id = transpose(reshape(1:N, L, L))
        agents_properties = rand(0:1, L, L, 2)
    end
    agents_coordmapper = Dict(i => (div(i-1, L)+1, rem(i-1, L)+1) for i in 1:N);
    distance_matrix = [euclidean(agents_coordmapper[i], agents_coordmapper[j]) for i in 1:N, j in 1:N]
    impact = zeros((L, L, 3))
    # display("iteration = 0")
    # display(opinions)
    # display(agents_id)
    # display(agents_properties)
    # display(agents_coordmapper)
    # display(distance_matrix)
    # display(impact)

    zad==1 ? show_grid(t, opinions, 12) : show_grid(t, opinions, 3)
    while length(unique(opinions)) != 1 && t<11
        points_grouped = [findall(x -> x == i, opinions) for i in 1:3]
        points_grouped_ids = [[agents_id[idx] for idx in group] for group in points_grouped]
        
        for i in 1:N
            curr_point_cords = agents_coordmapper[i]
            curr_point_opinion = opinions[curr_point_cords[1], curr_point_cords[2]]
            for opinion_type in [1, 2, 3]
                sum = 0
                for (point, point_id)  in zip(points_grouped[opinion_type], points_grouped_ids[opinion_type])
                    if (curr_point_opinion == opinion_type)
                        sum += ((agents_properties[point[1], point[2], 1]) / (1 + distance_matrix[i, point_id]^alpha))
                    else
                        sum += ((agents_properties[point[1], point[2], 2]) / (1 + distance_matrix[i, point_id]^alpha))
                    end
                end
                impact[curr_point_cords[1], curr_point_cords[2], opinion_type] = 4*sum
            end
        end
        opinions = [argmax(impact[i, j, :]) for i in 1:L, j in 1:L]
        
        t+=1
        zad==1 ? show_grid(t, opinions, 12) : show_grid(t, opinions, 3)
        # display("------------------------------")
        # display("interation = $t")
        # display(opinions)
        # display(impact)
        # display(new_opinions)
        # opinions = new_opinions
    end
end

NSL_model (generic function with 1 method)

In [5]:
using Plots

fixed_colors = [:tomato, :mediumseagreen, :steelblue]

function show_grid(t, opinions, font, alpha, L)
    mapped_values = [op == 1 ? 1 : (op == 2 ? 2 : 3) for op in opinions]

    p = heatmap(mapped_values, color=fixed_colors, xticks=:none, yticks=:none,
                clim=(1,3), legend=false, frame=false, yflip=true, title="L=$L, alpha=$alpha, t=$t",
                size=(1920, 1080), titlefontsize=32)

    counter = 1
    for i in 1:size(opinions, 1), j in 1:size(opinions, 2)
        annotate!(j, i, text(string(counter), :black, :center, font))
        counter += 1
    end
    return p  # Return the plot instead of displaying it
end

function NSL_model(; L=nothing, alpha=nothing, zad=nothing, fps=nothing)
    N = L^2
    t = 0
    frames = []  # Store frames for animation

    if (zad == 1)
        opinions = [1 2 1;
                    1 3 3;
                    1 2 3];
        agents_id = [1 2 3;
                     4 5 6;
                     7 8 9];
        agents_properties = permutedims(reshape([(i/10, 1 - i/10)[k] for i in 1:N, k in 1:2], 3, 3, 2), (2, 1, 3))
    elseif (zad == 2)
        opinions = rand(1:3, L, L)
        agents_id = transpose(reshape(1:N, L, L))
        agents_properties = rand(0:1, L, L, 2)
    end
    agents_coordmapper = Dict(i => (div(i-1, L)+1, rem(i-1, L)+1) for i in 1:N)
    distance_matrix = [euclidean(agents_coordmapper[i], agents_coordmapper[j]) for i in 1:N, j in 1:N]
    impact = zeros((L, L, 3))

    prev_opinions1 = deepcopy(opinions)  # Stan z poprzedniej iteracji
    prev_opinions2 = deepcopy(opinions)  # Stan sprzed dwóch iteracji

    push!(frames, zad == 1 ? show_grid(t, opinions, 100, alpha, L) : show_grid(t, opinions, 20, alpha, L))

    while length(unique(opinions)) != 1 && t < 50
        points_grouped = [findall(x -> x == i, opinions) for i in 1:3]
        points_grouped_ids = [[agents_id[idx] for idx in group] for group in points_grouped]

        for i in 1:N
            curr_point_cords = agents_coordmapper[i]
            curr_point_opinion = opinions[curr_point_cords[1], curr_point_cords[2]]
            for opinion_type in [1, 2, 3]
                sum = 0
                for (point, point_id) in zip(points_grouped[opinion_type], points_grouped_ids[opinion_type])
                    if (curr_point_opinion == opinion_type)
                        sum += ((agents_properties[point[1], point[2], 1]) / (1 + distance_matrix[i, point_id]^alpha))
                    else
                        sum += ((agents_properties[point[1], point[2], 2]) / (1 + distance_matrix[i, point_id]^alpha))
                    end
                end
                impact[curr_point_cords[1], curr_point_cords[2], opinion_type] = 4*sum
            end
        end
        new_opinions = [argmax(impact[i, j, :]) for i in 1:L, j in 1:L]

        t += 1

        push!(frames, zad == 1 ? show_grid(t, new_opinions, 100, alpha, L) : show_grid(t, new_opinions, 20, alpha, L))

         # Jeśli nowy stan = stan sprzed dwóch iteracji -> zatrzymujemy symulację
        if new_opinions == prev_opinions2
            println("🔁 Opinie fluktuują cyklicznie! Symulacja zatrzymana na t=$t")
            break
        end

        # Aktualizujemy stany dla kolejnych iteracji
        prev_opinions2 = deepcopy(prev_opinions1)
        prev_opinions1 = deepcopy(opinions)
        opinions = new_opinions

        
    end

    anim = @animate for frame in frames
        plot(frame)
    end
    gif(anim, "animations/zad2_L$(L)_alpha$(alpha).mp4", fps=fps, show_msg=false)  # Save animation
end

NSL_model (generic function with 1 method)

In [6]:
NSL_model(L=3, alpha=2, zad=1, fps=1)

Plots.AnimatedGif("/Users/stawager/Repos/socjologiaobliczeniowa2025/zad2/animations/zad2_L3_alpha2.mp4")

In [7]:
NSL_model(L=3, alpha=3, zad=1, fps=1)

🔁 Opinie fluktuują cyklicznie! Symulacja zatrzymana na t=2


Plots.AnimatedGif("/Users/stawager/Repos/socjologiaobliczeniowa2025/zad2/animations/zad2_L3_alpha3.mp4")

___

In [8]:
NSL_model(L=20, alpha=1, zad=2, fps=1)

Plots.AnimatedGif("/Users/stawager/Repos/socjologiaobliczeniowa2025/zad2/animations/zad2_L20_alpha1.mp4")

In [9]:
NSL_model(L=20, alpha=2, zad=2, fps=2)

Plots.AnimatedGif("/Users/stawager/Repos/socjologiaobliczeniowa2025/zad2/animations/zad2_L20_alpha2.mp4")

In [10]:
NSL_model(L=20, alpha=3, zad=2, fps=3)

Plots.AnimatedGif("/Users/stawager/Repos/socjologiaobliczeniowa2025/zad2/animations/zad2_L20_alpha3.mp4")

In [11]:
NSL_model(L=20, alpha=4, zad=2, fps=3)

Plots.AnimatedGif("/Users/stawager/Repos/socjologiaobliczeniowa2025/zad2/animations/zad2_L20_alpha4.mp4")